<h4><center>This code determines the average growing season for each watershed based on the last day of below -1 temps in the spring and the first day of below -1 temps in the fall.

### Packages to Import

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import rasterio
import fiona
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.plot import show
import rasterio.mask
import datetime 
import gdal

### Function for creating rasters from the .nc file data

In [4]:
def CreateRaster(lon,lat,data,resx,resy,save_directory):
    from osgeo import gdal
    from osgeo import gdal_array
    from osgeo import osr,ogr

    xmin,ymin,xmax,ymax = [lon.min(),lat.min(),lon.max(),lat.max()]
    nrows,ncols = np.shape(data)
    geotransform=(xmin-resy,resy,0,ymax+resx,0, -resx)  

    output_raster = gdal.GetDriverByName('GTiff').Create(save_directory,ncols, nrows, 1 ,gdal.GDT_Float32)  # Open the file

    output_raster.SetGeoTransform(geotransform)  # Specify its coordinates
    srs = osr.SpatialReference()                 # Establish its coordinate encoding
    srs.ImportFromEPSG(4326)                     # This one specifies WGS84 lat long.

    output_raster.SetProjection( srs.ExportToWkt() )   # Exports the coordinate system                                         
    output_raster.GetRasterBand(1).WriteArray(data)   # Writes my array to the raster
    output_raster.FlushCache()
    return

In [14]:
Less_then[0]

0

In [26]:
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/'
vari='t2m' #temperature variable

#loop through for stat and end dates
for Time in ['Start','End']:
    for year in np.arange(1981,2019,1):
        TB=pd.read_pickle(path+'Temp/Daily_Vars/%s_%s_MIN.pkl'%(vari,year)) #Data containing min daily temps
        HOLD=[]
        if Time=='Start':
            for i in np.arange(0,160):    # loop through min temps for each day until mid-June (i is day of year)
                HOLD.append(TB[TB.index==i].Var.values[0])
        if Time=='End':
            for i in np.arange(250,360):  #loop through min temps for each day after mid-September (i is day of year)
                HOLD.append(TB[TB.index==i].Var.values[0])
        HOLD=np.array(HOLD)

        #Initilize growing season array --> each grid cell will be filled with the first or last freezing day
        Shape= np.shape(HOLD)
        Growing_Season=np.empty([Shape[1],Shape[2]])
        
        #loop though each grid cell
        for i in np.arange(0,Shape[1]):
            for j in np.arange(0,Shape[2]):
                
                # select all minimum values in a given gridcell
                Array=HOLD[:,i,j]
                
                #check if minimum value is nan
                if np.isnan(np.min(Array))==True:
                    Growing_Season[i,j]=np.NAN
                    continue
                
                #determine where minimum values are less than -1
                Less_then=np.where(HOLD[:,i,j]<=-1)[0]
                
                #make sure grid cell has at least 9 days where it is freezing --> if false set grid cell start/end to Nan
                if len(Less_then)<10:
                    Growing_Season[i,j]=np.NaN
                    continue
                
                #insures that the -1 degree temp is not anomalous (helps select start of sustained freezing)
                if Time=='End':
                    X=0; PICK=0
                    while X==0:
                        #select three consecutibe values in Less_then array (first time its the first three)
                        K= Less_then[PICK]
                        K1= Less_then[PICK+1]
                        K2= Less_then[PICK+2]

                        #check how many days difference is between them 
                        Diff=K-K1
                        Diff2=K1-K2

                        #if the first and second -1 day are more than 4 days apart and 
                        #      the second and third days are more than 2 days apart we will move onto 
                        #      the next three consecutive entires
                        if (Diff >4) &(Diff2 >2):
                            PICK=PICK+1
                            continue
                        X= 1
                        
                    #puts end of growing season day of year in raster
                    Growing_Season[i,j]=250+K #250 is day of year minimum for growing season end

                #insures that the -1 degree temp is not anomalous (helps select start of sustained freezing)
                if Time=='Start':
                    X=0; PICK=-1
                    while X==0:
                        #select three consecutibe values in Less_then array (first time its the first three)
                        K= Less_then[PICK]
                        K1= Less_then[PICK-1]
                        K2= Less_then[PICK-2]

                        #check how many days difference is between them 
                        Diff=K-K1
                        Diff2=K1-K2

                        #if the first and second -1 day are more than 4 days apart and 
                        #      the second and third days are more than 2 days apart we will move onto 
                        #      the next three consecutive entires
                        if (Diff >4) &(Diff2 >2):
                            PICK=PICK-1
                            continue
                        X= 1
                        
                    #puts start of growing season day of year in raster
                    Growing_Season[i,j]=K

                    
        #saves each years start and end of growing season raster 
        np.save(path+ 'MPB_Constraints/%s_GS/%s'%(Time,year),Growing_Season)


### Finds average start and end of growing season for each grid cell

In [24]:
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/MPB_Constraints/'

Start=[];End=[]

# for each year append the start and end into 1 file
for year in np.arange(1981,2019,1):
    Start.append(np.load(path+'Start_GS/%s.npy'%year))
    End.append(np.load(path+'End_GS/%s.npy'%year))
    
#create array of average start and end days of growing season
Avg_Start= np.nanmean(Start,axis=0)
Avg_End= np.nanmean(End,axis=0)

C:\Anaconda3\envs\Trial\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice
  import sys
C:\Anaconda3\envs\Trial\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  


### Convert arrays of avergae growing season to rasters 

In [17]:
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/MPB_Constraints/'

lat=np.load(path+'Lat.npy')
long=np.load(path+'Long.npy')

Resx= 0.1 #x resolution
Resy= 0.1 #y resolution
resample_value=6 # how much to decrease resolution by in resampling

Time=['Start','End']
for T in Time:
    if T=='Start'
        x=Avg_Start
    if T=='End':
        x=Avg_End
        
    hold=[]
    Z_ERA5=[]
    
    # attach data to lat and long coordiantes in an arry
    for i in np.arange(0,len(lat)):
        for j in np.arange(0,len(long)):
            Z_ERA5.append([x[i][j],float(lat[i]),float(long[j])])
    hold.append(Z_ERA5) 



    #extract raster array to work with
    working=hold[0]

    #used for reshaping arrays
    shp=len(np.unique(np.array(working)[:,2]))

    #where to save the raster to
    Save_file=path+'Rasters/GS.tif'

    # reshape data dimensions
    Data_array_ERA5=np.reshape(np.array(working)[:,0],(-1,shp))
    lat_array_ERA5=np.reshape(np.array(working)[:,1], (-1,shp))
    lon_array_ERA5=np.reshape(np.array(working)[:,2],(-1,shp))

    # Create the raster
    CreateRaster(lon=lon_array_ERA5,lat=lat_array_ERA5,data=Data_array_ERA5, resx=Resx, resy=Resy, save_directory=Save_file)




    # where to save resmapled raster to
    if T='Start':
        Save_file_N=path+'Rasters/StartGrowingSeason.tif' 
    if T='End':
        Save_file_N=path+'Rasters/EndGrowingSeason.tif' 

    # resample raster 
    with rasterio.open(Save_file) as dataset:

        dataP = dataset.read(1, out_shape=(dataset.height * resample_value, dataset.width * resample_value))

        res_new = 0.1/resample_value
        lat_array_new_P=np.arange(lat_array_ERA5.max()+(res_new*(resample_value-1)),lat_array_ERA5.min()-0.001,-res_new)
        lat_array_new_P=np.array([lat_array_new_P]*(np.shape(dataP)[1])).transpose()

        lon_array_new_P=np.arange(lon_array_ERA5.max(),lon_array_ERA5.min()-(res_new*(resample_value-1))-0.001, -res_new)
        lon_array_new_P=np.array([lon_array_new_P]*(np.shape(dataP)[0]))


    CreateRaster(lon_array_new_P,lat_array_new_P,dataP,res_new, res_new, Save_file_N)
    
    #delete non-resampled raster
    gdal.GetDriverByName('GTiff').Delete(Save_file)

    #save the new lat and long coordiantes from resampling
    np.save(path+'Rasters/Lat_NEW',np.array(lat_array_new_P))
    np.save(path+'Rasters/Long_NEW',np.array(lon_array_new_P))

### Finding average growing season for each watershed

In [27]:
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/MPB_Constraints/'
watersheds=['Spius','Chilko','Chilcotin','Nation','Osilinka','Mesilinka','Stellako','Nautley']
Ras=['StartGrowingSeason','EndGrowingSeason']

#initiate result dataframe
Results=pd.DataFrame({'WS':watersheds})

#loop through for start and end rasters
for R in Ras:
    Avg=[]
    for WS in watersheds:
    
        #select shapefile for clipping of raster
        with fiona.open("D:/UNBC/HydroMet_Project/GIS_maps/Shapefiles_WaterFeatures/Individual_ws/%s/%s_WS.shp"%(WS,WS), "r") as shapefile:
            features = [feature["geometry"] for feature in shapefile]

        #clip raster to watershed area
        with rasterio.open(path+"Rasters/%s.tif"%R) as src:
            out_image, out_transform = rasterio.mask.mask(src, features,crop=True,nodata=np.nan)
            out_meta = src.meta.copy()
        
        #average start/end growing season day of year in watershed
        Avg.append(round(np.nanmean(out_image)))
    
    #save results for each watershed
    Results[R]=Avg
                       

In [28]:
Results

,WS,StartGrowingSeason,EndGrowingSeason
0,Spius,143.0,279.0
1,Chilko,162.0,265.0
2,Chilcotin,149.0,269.0
3,Nation,141.0,276.0
4,Osilinka,153.0,262.0
5,Mesilinka,156.0,262.0
6,Stellako,137.0,289.0
7,Nautley,134.0,289.0


###### I create a excel sheet with the results containing the day of year,  month, and day of each of the averages